In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import lightgbm as lgbm
import gc
import xgboost as xgb
import pickle as pickle


from catboost import CatBoostRegressor
import dask.dataframe as dd
from sklearn.model_selection import KFold

In [43]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [44]:
y_cols = ['shop_id', 'item_id', 'date_block_num']

In [45]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'] != 2013]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [46]:
train_item_ids = sales_train['item_id'].unique()
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [47]:
combinations = []
for shop in all_shop_ids:
    #get all article ids ever associated to this shop
    train_ids = sales_train[sales_train['shop_id'] == shop]['item_id'].unique()
    test_ids = sales_train[sales_train['shop_id'] == shop]['item_id'].unique()
    all_shop = np.unique(np.append(train_ids, test_ids))
    all_shop_combo = [[shop, item, block] for item in all_shop for block in train_blocks]
    for combo in all_shop_combo:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])
del combinations

In [48]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"y"})

In [ ]:
#

In [320]:
training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)
training = training.apply(pd.to_numeric,downcast='unsigned')

training['y'] = training['y'].clip(0,20)
training['y'] = training['y'].astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

In [325]:
def get_mean_encoding(df, group_cols, target):
    cumsum = df.groupby(group_cols)[target].cumsum() - df[target]
    cumcnt = df.groupby(group_cols).cumcount()
    return cumsum/cumcnt

training['item_me'] = pd.to_numeric(get_mean_encoding(training, ['item_id'], 'y'), downcast='float')
training['shop_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id'], 'y'), downcast='float')
training['category_me'] = pd.to_numeric(get_mean_encoding(training, ['item_category_id'], 'y'), downcast='float')
training['shop_category_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id', 'item_category_id'], 'y'), downcast='float')
training['shop_item_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id', 'item_id'], 'y'), downcast='float')

training.fillna(0,inplace=True)

In [321]:
cols = ['item_id','date_block_num']
item_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'item_block_units'})

training = training.merge(item_block_units, on=cols, how='left').fillna(0)
del item_block_units
gc.collect()
#
cols = ['shop_id','date_block_num']
shop_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'shop_block_units'})

training = training.merge(shop_block_units, on=cols, how='left').fillna(0)
del shop_block_units
gc.collect()
#
cols = ['item_category_id','date_block_num']
cat_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'cat_block_units'})

training = training.merge(cat_block_units, on=cols, how='left').fillna(0)
del cat_block_units
gc.collect()
#

cols = ['shop_id', 'item_category_id','date_block_num']
shop_cat_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'shop_cat_block_units'})

training = training.merge(shop_cat_block_units, on=cols, how='left').fillna(0)
del shop_cat_block_units
gc.collect()
#
cols = ['shop_id', 'item_id','date_block_num']
shop_item_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'shop_item_block_units'})

training = training.merge(shop_item_block_units, on=cols, how='left').fillna(0)
del shop_item_block_units
gc.collect()

63

In [352]:
number_of_items = sales_train['item_id'].nunique()
print("number_of_items:", number_of_items)
number_of_categories = sales_train['item_category_id'].nunique()
print("number_of_categories:", number_of_categories)
number_of_shops = sales_train['shop_id'].nunique()
print("number_of_shops:", number_of_shops)
number_of_days = 365 + 365 - 30 - 31
print("number_of_days:", number_of_days)
number_of_blocks = sales_train['date_block_num'].nunique()
print("number_of_blocks:", number_of_blocks)
total_sales = sales_train['item_cnt_day'].sum()
print("total_sales:", total_sales)
average_price = sales_train['item_price'].mean()
print("average_price:", average_price)

number_of_items: 17054
number_of_categories: 79
number_of_shops: 55
number_of_days: 669
number_of_blocks: 22
total_sales: 2085473.0
average_price: 1015.5023073772021


In [355]:
training['item_share_of_total_units'] = training['item_units'] * 100 / total_sales
training['category_share_of_total_units'] = training['cat_units'] * 100 / total_sales
training['shop_share_of_units'] = training['shop_units'] * 100 / total_sales
training['shop_item_share_of_total_units'] = training['shop_item_units'] * 100\
                        / total_sales
training['shop_item_share_of_shop_units'] = training['shop_item_units'] * 100\
                        / training['shop_units']

In [337]:
training['item_units'] = training.groupby(['date_block_num'])['item_block_units'].transform(np.sum)
training['item_max_units_block'] = training.groupby(['item_id'])['item_block_units']\
        .transform(np.max)
training['item_min_units_block'] = training.groupby(['item_id'])['item_block_units']\
        .transform(np.min)
training['item_minmax_mean'] = training[['item_max_units_block', 'item_min_units_block']].mean(axis=1)

#
training['shop_units'] = training.groupby(['date_block_num'])['shop_block_units'].transform(np.sum)
training['shop_max_units_block'] = training.groupby(['shop_id'])['shop_block_units']\
        .transform(np.max)
training['shop_min_units_block'] = training.groupby(['shop_id'])['shop_block_units']\
        .transform(np.min)
training['shop_minmax_mean'] = training[['shop_max_units_block', 'shop_min_units_block']].mean(axis=1)

#
training['cat_units'] = training.groupby(['date_block_num'])['cat_block_units'].transform(np.sum)
training['cat_max_units_block'] = training.groupby(['item_category_id'])['cat_block_units']\
        .transform(np.max)
training['cat_min_units_block'] = training.groupby(['item_category_id'])['cat_block_units']\
        .transform(np.min)
training['cat_minmax_mean'] = training[['cat_max_units_block', 'cat_min_units_block']].mean(axis=1)
#
training['shop_cat_units'] = training.groupby(['date_block_num'])['shop_cat_block_units'].transform(np.sum)
training['shop_cat_max_units_block'] = training.groupby(['shop_id', 'item_category_id'])['shop_cat_block_units']\
        .transform(np.max)
training['shop_cat_min_units_block'] = training.groupby(['shop_id', 'item_category_id'])['shop_cat_block_units']\
        .transform(np.min)
training['shop_cat_minmax_mean'] = training[['shop_cat_max_units_block', 'shop_cat_min_units_block']].mean(axis=1)
#
training['shop_item_units'] = training.groupby(['date_block_num'])['shop_item_block_units'].transform(np.sum)
training['shop_item_max_units_block'] = training.groupby(['shop_id', 'item_id'])['shop_item_block_units']\
        .transform(np.max)
training['shop_item_min_units_block'] = training.groupby(['shop_id', 'item_id'])['shop_item_block_units']\
        .transform(np.min)
training['shop_item_minmax_mean'] = training[['shop_item_max_units_block', 'shop_item_min_units_block']].mean(axis=1)

In [349]:
training[['item_units','shop_units']].sample(10)

,item_units,shop_units
1177846,5112241.0,770404514.0
384032,4624188.0,793683070.0
4043938,4320601.0,651577325.0
1505571,4498404.0,738502367.0
4060820,2926344.0,477760865.0
2010321,4156779.0,652820532.0
3235962,4146610.0,646954534.0
4596673,3140531.0,514980688.0
594362,5112241.0,770404514.0
2525744,4698631.0,718360068.0


In [322]:
rolls = [3,6]
cols = ['item_id','date_block_num']

for roll in rolls:
    print(roll)
    roll_name = "item_block_units_rolling_" + str(roll)
    roll_name_tmp = roll_name + "_tmp"
    
    item_block_units_rolling_temp = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        ['item_block_units'].rolling(roll,min_periods=2).mean().reset_index()\
        .rename(columns={'item_block_units':roll_name_tmp})\
        [['item_id','date_block_num',roll_name_tmp]]
    
    training = training.merge(item_block_units_rolling_temp, on=cols, how='left')

    item_block_units_rolling = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        [roll_name_tmp].shift(1)\
        .rename(columns={roll_name_tmp:roll_name}).reset_index()

    training = training.merge(item_block_units_rolling, on=cols, how='left')
    training.drop(columns=[roll_name_tmp], inplace=True)

3
6


In [346]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'y', 'item_category_id',
       'item_block_units', 'shop_block_units', 'cat_block_units',
       'shop_cat_block_units', 'shop_item_block_units',
       'item_block_units_rolling_3', 'item_block_units_rolling_6',
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
       'shop_item_me', 'val_ignore', 'item_units', 'item_max_units_block',
       'item_min_units_block', 'shop_units', 'shop_max_units_block',
       'shop_min_units_block', 'cat_units', 'cat_max_units_block',
       'cat_min_units_block', 'shop_cat_units',
       'shop_cat_max_units_block', 'shop_cat_min_units_block',
       'shop_item_units', 'shop_item_max_units_block',
       'shop_item_min_units_block', 'item_minmax_mean',
       'shop_minmax_mean', 'cat_minmax_mean', 'shop_cat_minmax_mean',
       'shop_item_minmax_mean'], dtype=object)

In [196]:
np.min([2,2])

2

In [23]:
training['rolling_composite'] =  training['shop_block_units_rolling3'].clip(1, None) *\
            training['item_block_units_rolling3'].clip(1, None) 
training['me_composite'] =  training['item_me'].clip(1, None)  * training['shop_me'].clip(1, None) 

In [52]:
training.dtypes

item_id                    int64
shop_id                    uint8
date_block_num             uint8
y                           int8
item_category_id           int64
item_me                  float32
shop_me                  float32
category_me              float32
shop_category_me         float32
shop_item_me             float32
item_block_units         float64
shop_block_units         float64
cat_block_units          float64
shop_cat_block_units     float64
shop_item_block_units    float64
dtype: object

In [55]:
training.sample(10)

,item_id,shop_id,date_block_num,y,item_category_id,item_me,shop_me,category_me,shop_category_me,shop_item_me,item_block_units,shop_block_units,cat_block_units,shop_cat_block_units,shop_item_block_units,item_units,item_max_units_block,item_min_units_block
650693,2916,49,33,0,23,0.308470,0.217947,0.313670,0.211524,0.333333,0.0,648.0,2203.0,42.0,0.0,2499996.0,71.0,0.0
1680959,6608,19,17,0,25,0.040816,0.434715,0.180928,0.122460,0.000000,0.0,2067.0,460.0,17.0,0.0,4156779.0,6.0,0.0
3105180,12080,38,24,0,40,0.155172,0.369399,0.239276,0.121243,0.000000,0.0,1441.0,15109.0,76.0,0.0,4698631.0,30.0,0.0
2142666,8401,28,32,0,37,0.269710,0.715043,0.181585,0.367775,0.650000,0.0,2979.0,2989.0,158.0,0.0,2528824.0,47.0,0.0
114587,1021,7,23,1,67,0.151515,0.219186,0.195883,0.159780,0.727273,15.0,3318.0,2778.0,49.0,1.0,7399317.0,18.0,0.0
4914915,18159,31,17,0,40,0.111111,0.686598,0.229800,0.635294,0.400000,1.0,7701.0,16064.0,1835.0,0.0,4156779.0,4.0,0.0
2916352,11329,31,22,0,43,0.256098,0.740620,0.183567,0.555323,1.400000,6.0,9865.0,614.0,183.0,0.0,5112241.0,9.0,0.0
4491899,16628,27,17,0,40,0.372549,0.494315,0.232576,0.246279,0.000000,0.0,4282.0,16064.0,390.0,0.0,4156779.0,146.0,0.0
3687409,14174,52,23,0,43,0.072727,0.242092,0.189033,0.097643,0.181818,0.0,2243.0,740.0,11.0,0.0,7399317.0,4.0,0.0
4218749,15811,59,19,1,55,0.306122,0.265957,0.224547,0.150501,0.000000,20.0,1244.0,11180.0,125.0,1.0,4474489.0,23.0,5.0


In [314]:
print(np.mean(np.array([31,24,58])))
print(np.mean(np.array([15,14,11])))
print(np.mean(np.array([6,9,53])))

37.666666666666664
13.333333333333334
22.666666666666668


In [324]:
training[(training['item_id'].isin([30,31])) & (training['shop_id'] == 30)]\
        .sort_values(['item_id','date_block_num'])[['item_id','shop_id',\
                                                    'date_block_num','item_block_units',\
                                                    'item_block_units_rolling_3', 'item_block_units_rolling_6']]
                                                    #'item_block_units_rolling_6']]

,item_id,shop_id,date_block_num,item_block_units,item_block_units_rolling_3,item_block_units_rolling_6
1144,30,30,12,58.0,NaN,NaN
1145,30,30,13,24.0,NaN,NaN
1146,30,30,14,31.0,41.000000,41.000000
1147,30,30,15,21.0,37.666667,37.666667
1148,30,30,16,16.0,25.333333,33.500000
1149,30,30,17,13.0,22.666667,30.000000
1150,30,30,18,13.0,16.666667,27.166667
1151,30,30,19,12.0,14.000000,19.666667
1152,30,30,20,11.0,12.666667,17.666667
1153,30,30,21,13.0,12.000000,14.333333


In [35]:
len(training[training['item_block_units'] > 0])

3456135

In [37]:
len(transactions_items_blocks)

135451

In [360]:
gc.collect()
val = training[training['date_block_num'] == 33]
print("val length", len(val))

unique_pairs_val = list(set(list(zip(val.shop_id, val.item_id))))
print("number of unique shop/item pairs in val", len(unique_pairs_val))
unique_pairs_val_ignore = unique_pairs_val[0:int(len(unique_pairs_val)/2)]


def tuple2key(t):
    return "%d_%d" % (t[0], t[1])

val_pairs_ignore_dict = {}
for t in unique_pairs_val_ignore:
    val_pairs_ignore_dict[tuple2key(t)] = 1
 
    
training['val_ignore'] = (training['shop_id'].astype(str) + '_' +  training['item_id'].astype(str))\
                                    .apply(lambda x: x in val_pairs_ignore_dict)

val length 270742
number of unique shop/item pairs in val 270742


In [101]:
len(training[training['val_ignore'] == True])

2978162

In [361]:
gc.collect()


x_train = training[(training['date_block_num'] < 33) & (training['val_ignore'] == False)]
y_train = x_train['y']


pos_train_len = len(y_train[y_train != 0])
print("pos_train_len", pos_train_len)

zeros_keep_indices_train = y_train[y_train == 0].sample(int(pos_train_len/6)).index
print("zeros_keep_indices_train", len(zeros_keep_indices_train))
non_zeros_train_indices = y_train[y_train != 0].index
print("non_zeros_train_indices", len(non_zeros_train_indices))

train_indices = np.append(np.array(zeros_keep_indices_train), np.array(non_zeros_train_indices))

y_train = y_train.loc[train_indices]
x_train = x_train.loc[train_indices]




x_val = training[training['date_block_num'] == 33]
y_val = x_val['y']

pos_val_len = len(y_val[y_val != 0])
print("pos_val_len", pos_val_len)

zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/6)).index
print("zeros_keep_indices_val", len(zeros_keep_indices_val))
non_zeros_val_indices = y_val[y_val != 0].index
print("non_zeros_val_indices", len(non_zeros_val_indices))

val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))

y_val = y_val.loc[val_indices]
x_val = x_val.loc[val_indices]

pos_train_len 444760
zeros_keep_indices_train 74126
non_zeros_train_indices 444760
pos_val_len 31471
zeros_keep_indices_val 5245
non_zeros_val_indices 31471


In [357]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'y', 'item_category_id',
       'item_block_units', 'shop_block_units', 'cat_block_units',
       'shop_cat_block_units', 'shop_item_block_units',
       'item_block_units_rolling_3', 'item_block_units_rolling_6',
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
       'shop_item_me', 'val_ignore', 'item_units', 'item_max_units_block',
       'item_min_units_block', 'shop_units', 'shop_max_units_block',
       'shop_min_units_block', 'cat_units', 'cat_max_units_block',
       'cat_min_units_block', 'shop_cat_units',
       'shop_cat_max_units_block', 'shop_cat_min_units_block',
       'shop_item_units', 'shop_item_max_units_block',
       'shop_item_min_units_block', 'item_minmax_mean',
       'shop_minmax_mean', 'cat_minmax_mean', 'shop_cat_minmax_mean',
       'shop_item_minmax_mean', 'item_share_of_total_units',
       'category_share_of_total_units', 'shop_share_of_units',
       'shop_item_share_of_total_units', 'shop_i

In [358]:



cb_features = [
      
       'item_block_units_rolling_3', 'item_block_units_rolling_6',
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
       'shop_item_me', 'val_ignore', 'item_units', 'item_max_units_block',
       'item_min_units_block', 'shop_units', 'shop_max_units_block',
       'shop_min_units_block', 'cat_units', 'cat_max_units_block',
       'cat_min_units_block', 'shop_cat_units',
       'shop_cat_max_units_block', 'shop_cat_min_units_block',
       'shop_item_units', 'shop_item_max_units_block',
       'shop_item_min_units_block', 'item_minmax_mean',
       'shop_minmax_mean', 'cat_minmax_mean', 'shop_cat_minmax_mean',
       'shop_item_minmax_mean','item_share_of_total_units',
       'category_share_of_total_units', 'shop_share_of_units',
       'shop_item_share_of_total_units', 'shop_item_share_of_shop_units']

In [365]:
cb_model = CatBoostRegressor(iterations=70000,
                             learning_rate=0.01,
                             eval_metric='RMSE',
                             #thread_count=16,
                             task_type = "GPU",
                             use_best_model=True,
                             #l2_leaf_reg = 1000,
                             od_type = "Iter",
                             od_wait = 30,
                             #random_strength = 10,
                             #bagging_temperature = 1,
                             #one_hot_max_size = 2,
                             random_seed = 42)

#drops = ['subcategory','area']
#x_train = x_train.drop(columns=drops)
#x_val = x_val.drop(columns=drops)


cb_model.fit(x_train[cb_features], y_train, #cat_features=categorical_features_indices,
             eval_set=(x_val[cb_features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

0:	learn: 2.9634625	test: 2.9494810	best: 2.9494810 (0)	total: 6.87ms	remaining: 8m 1s
1:	learn: 2.9445812	test: 2.9313396	best: 2.9313396 (1)	total: 12.2ms	remaining: 7m 6s
2:	learn: 2.9261681	test: 2.9151390	best: 2.9151390 (2)	total: 18.1ms	remaining: 7m 1s
3:	learn: 2.9078453	test: 2.8977065	best: 2.8977065 (3)	total: 23.3ms	remaining: 6m 48s
4:	learn: 2.8899257	test: 2.8819230	best: 2.8819230 (4)	total: 28.6ms	remaining: 6m 40s
5:	learn: 2.8720639	test: 2.8645751	best: 2.8645751 (5)	total: 34ms	remaining: 6m 36s
6:	learn: 2.8544419	test: 2.8473868	best: 2.8473868 (6)	total: 39.2ms	remaining: 6m 32s
7:	learn: 2.8370149	test: 2.8305893	best: 2.8305893 (7)	total: 44.4ms	remaining: 6m 28s
8:	learn: 2.8198168	test: 2.8139254	best: 2.8139254 (8)	total: 49.7ms	remaining: 6m 26s
9:	learn: 2.8028685	test: 2.7967406	best: 2.7967406 (9)	total: 54.9ms	remaining: 6m 24s
10:	learn: 2.7862134	test: 2.7795776	best: 2.7795776 (10)	total: 60.1ms	remaining: 6m 22s
11:	learn: 2.7697316	test: 2.763571

110:	learn: 1.8897366	test: 1.8673342	best: 1.8673342 (110)	total: 599ms	remaining: 6m 17s
111:	learn: 1.8858251	test: 1.8629808	best: 1.8629808 (111)	total: 605ms	remaining: 6m 17s
112:	learn: 1.8819261	test: 1.8584761	best: 1.8584761 (112)	total: 610ms	remaining: 6m 17s
113:	learn: 1.8780801	test: 1.8543218	best: 1.8543218 (113)	total: 615ms	remaining: 6m 16s
114:	learn: 1.8743042	test: 1.8498910	best: 1.8498910 (114)	total: 620ms	remaining: 6m 16s
115:	learn: 1.8705783	test: 1.8455557	best: 1.8455557 (115)	total: 625ms	remaining: 6m 16s
116:	learn: 1.8669332	test: 1.8413740	best: 1.8413740 (116)	total: 631ms	remaining: 6m 16s
117:	learn: 1.8632878	test: 1.8371097	best: 1.8371097 (117)	total: 636ms	remaining: 6m 16s
118:	learn: 1.8597507	test: 1.8329290	best: 1.8329290 (118)	total: 643ms	remaining: 6m 17s
119:	learn: 1.8562793	test: 1.8285285	best: 1.8285285 (119)	total: 652ms	remaining: 6m 19s
120:	learn: 1.8528197	test: 1.8244502	best: 1.8244502 (120)	total: 658ms	remaining: 6m 19s

221:	learn: 1.6745933	test: 1.5968272	best: 1.5968272 (221)	total: 1.19s	remaining: 6m 12s
222:	learn: 1.6737488	test: 1.5955005	best: 1.5955005 (222)	total: 1.19s	remaining: 6m 12s
223:	learn: 1.6729069	test: 1.5942315	best: 1.5942315 (223)	total: 1.2s	remaining: 6m 12s
224:	learn: 1.6720083	test: 1.5927409	best: 1.5927409 (224)	total: 1.2s	remaining: 6m 12s
225:	learn: 1.6712279	test: 1.5912063	best: 1.5912063 (225)	total: 1.21s	remaining: 6m 12s
226:	learn: 1.6703409	test: 1.5897741	best: 1.5897741 (226)	total: 1.21s	remaining: 6m 12s
227:	learn: 1.6695303	test: 1.5883975	best: 1.5883975 (227)	total: 1.22s	remaining: 6m 12s
228:	learn: 1.6687584	test: 1.5871393	best: 1.5871393 (228)	total: 1.22s	remaining: 6m 12s
229:	learn: 1.6679675	test: 1.5859298	best: 1.5859298 (229)	total: 1.23s	remaining: 6m 12s
230:	learn: 1.6672738	test: 1.5848749	best: 1.5848749 (230)	total: 1.23s	remaining: 6m 12s
231:	learn: 1.6665642	test: 1.5831902	best: 1.5831902 (231)	total: 1.24s	remaining: 6m 11s
2

336:	learn: 1.6172169	test: 1.5092068	best: 1.5092068 (336)	total: 1.78s	remaining: 6m 7s
337:	learn: 1.6168891	test: 1.5081452	best: 1.5081452 (337)	total: 1.78s	remaining: 6m 7s
338:	learn: 1.6166045	test: 1.5076624	best: 1.5076624 (338)	total: 1.79s	remaining: 6m 7s
339:	learn: 1.6163969	test: 1.5075286	best: 1.5075286 (339)	total: 1.79s	remaining: 6m 7s
340:	learn: 1.6161425	test: 1.5073839	best: 1.5073839 (340)	total: 1.8s	remaining: 6m 6s
341:	learn: 1.6158014	test: 1.5057662	best: 1.5057662 (341)	total: 1.8s	remaining: 6m 6s
342:	learn: 1.6154479	test: 1.5056226	best: 1.5056226 (342)	total: 1.81s	remaining: 6m 6s
343:	learn: 1.6151594	test: 1.5050686	best: 1.5050686 (343)	total: 1.81s	remaining: 6m 6s
344:	learn: 1.6148659	test: 1.5047896	best: 1.5047896 (344)	total: 1.82s	remaining: 6m 6s
345:	learn: 1.6145986	test: 1.5038880	best: 1.5038880 (345)	total: 1.82s	remaining: 6m 6s
346:	learn: 1.6142802	test: 1.5030909	best: 1.5030909 (346)	total: 1.83s	remaining: 6m 6s
347:	learn: 

450:	learn: 1.5894603	test: 1.4590831	best: 1.4590831 (450)	total: 2.36s	remaining: 6m 4s
451:	learn: 1.5892706	test: 1.4590188	best: 1.4590188 (451)	total: 2.37s	remaining: 6m 4s
452:	learn: 1.5891261	test: 1.4588558	best: 1.4588558 (452)	total: 2.37s	remaining: 6m 4s
453:	learn: 1.5889327	test: 1.4588714	best: 1.4588558 (452)	total: 2.38s	remaining: 6m 4s
454:	learn: 1.5887764	test: 1.4585813	best: 1.4585813 (454)	total: 2.38s	remaining: 6m 4s
455:	learn: 1.5885663	test: 1.4583543	best: 1.4583543 (455)	total: 2.39s	remaining: 6m 4s
456:	learn: 1.5884287	test: 1.4582798	best: 1.4582798 (456)	total: 2.39s	remaining: 6m 4s
457:	learn: 1.5882523	test: 1.4581953	best: 1.4581953 (457)	total: 2.4s	remaining: 6m 4s
458:	learn: 1.5880991	test: 1.4572345	best: 1.4572345 (458)	total: 2.4s	remaining: 6m 4s
459:	learn: 1.5878714	test: 1.4573427	best: 1.4572345 (458)	total: 2.41s	remaining: 6m 4s
460:	learn: 1.5877128	test: 1.4569393	best: 1.4569393 (460)	total: 2.41s	remaining: 6m 4s
461:	learn: 

...

2057:	learn: 1.4589823	test: 1.3080512	best: 1.3079956 (2055)	total: 10.6s	remaining: 5m 51s
2058:	learn: 1.4589609	test: 1.3080596	best: 1.3079956 (2055)	total: 10.6s	remaining: 5m 51s
2059:	learn: 1.4589343	test: 1.3080513	best: 1.3079956 (2055)	total: 10.7s	remaining: 5m 51s
2060:	learn: 1.4589044	test: 1.3080542	best: 1.3079956 (2055)	total: 10.7s	remaining: 5m 51s
2061:	learn: 1.4588488	test: 1.3080388	best: 1.3079956 (2055)	total: 10.7s	remaining: 5m 51s
2062:	learn: 1.4588036	test: 1.3080081	best: 1.3079956 (2055)	total: 10.7s	remaining: 5m 51s
2063:	learn: 1.4587676	test: 1.3080640	best: 1.3079956 (2055)	total: 10.7s	remaining: 5m 51s
2064:	learn: 1.4587520	test: 1.3080549	best: 1.3079956 (2055)	total: 10.7s	remaining: 5m 51s
2065:	learn: 1.4587048	test: 1.3080455	best: 1.3079956 (2055)	total: 10.7s	remaining: 5m 51s
2066:	learn: 1.4586876	test: 1.3080404	best: 1.3079956 (2055)	total: 10.7s	remaining: 5m 51s
2067:	learn: 1.4586720	test: 1.3080439	best: 1.3079956 (2055)	total: 1

2172:	learn: 1.4546747	test: 1.3044397	best: 1.3044114 (2169)	total: 11.2s	remaining: 5m 50s
2173:	learn: 1.4546375	test: 1.3043860	best: 1.3043860 (2173)	total: 11.2s	remaining: 5m 50s
2174:	learn: 1.4545955	test: 1.3043756	best: 1.3043756 (2174)	total: 11.2s	remaining: 5m 50s
2175:	learn: 1.4545674	test: 1.3042616	best: 1.3042616 (2175)	total: 11.3s	remaining: 5m 50s
2176:	learn: 1.4545043	test: 1.3043772	best: 1.3042616 (2175)	total: 11.3s	remaining: 5m 50s
2177:	learn: 1.4544844	test: 1.3043649	best: 1.3042616 (2175)	total: 11.3s	remaining: 5m 50s
2178:	learn: 1.4544295	test: 1.3045270	best: 1.3042616 (2175)	total: 11.3s	remaining: 5m 50s
2179:	learn: 1.4543858	test: 1.3045383	best: 1.3042616 (2175)	total: 11.3s	remaining: 5m 50s
2180:	learn: 1.4543300	test: 1.3046239	best: 1.3042616 (2175)	total: 11.3s	remaining: 5m 50s
2181:	learn: 1.4542978	test: 1.3046543	best: 1.3042616 (2175)	total: 11.3s	remaining: 5m 50s
2182:	learn: 1.4542470	test: 1.3046019	best: 1.3042616 (2175)	total: 1

2286:	learn: 1.4503688	test: 1.3017612	best: 1.3017476 (2285)	total: 11.8s	remaining: 5m 50s
2287:	learn: 1.4503353	test: 1.3017524	best: 1.3017476 (2285)	total: 11.8s	remaining: 5m 50s
2288:	learn: 1.4503040	test: 1.3017441	best: 1.3017441 (2288)	total: 11.8s	remaining: 5m 50s
2289:	learn: 1.4502909	test: 1.3016745	best: 1.3016745 (2289)	total: 11.8s	remaining: 5m 50s
2290:	learn: 1.4502383	test: 1.3016244	best: 1.3016244 (2290)	total: 11.8s	remaining: 5m 50s
2291:	learn: 1.4501883	test: 1.3014834	best: 1.3014834 (2291)	total: 11.8s	remaining: 5m 50s
2292:	learn: 1.4501393	test: 1.3014894	best: 1.3014834 (2291)	total: 11.9s	remaining: 5m 50s
2293:	learn: 1.4500969	test: 1.3014760	best: 1.3014760 (2293)	total: 11.9s	remaining: 5m 50s
2294:	learn: 1.4500567	test: 1.3013619	best: 1.3013619 (2294)	total: 11.9s	remaining: 5m 50s
2295:	learn: 1.4500205	test: 1.3013394	best: 1.3013394 (2295)	total: 11.9s	remaining: 5m 50s
2296:	learn: 1.4499652	test: 1.3013751	best: 1.3013394 (2295)	total: 1

2400:	learn: 1.4465195	test: 1.2978930	best: 1.2977714 (2395)	total: 12.4s	remaining: 5m 49s
2401:	learn: 1.4464577	test: 1.2978558	best: 1.2977714 (2395)	total: 12.4s	remaining: 5m 49s
2402:	learn: 1.4464366	test: 1.2978565	best: 1.2977714 (2395)	total: 12.4s	remaining: 5m 49s
2403:	learn: 1.4463998	test: 1.2978121	best: 1.2977714 (2395)	total: 12.4s	remaining: 5m 49s
2404:	learn: 1.4463823	test: 1.2977800	best: 1.2977714 (2395)	total: 12.4s	remaining: 5m 49s
2405:	learn: 1.4463403	test: 1.2977572	best: 1.2977572 (2405)	total: 12.4s	remaining: 5m 49s
2406:	learn: 1.4462981	test: 1.2976167	best: 1.2976167 (2406)	total: 12.4s	remaining: 5m 49s
2407:	learn: 1.4462656	test: 1.2976306	best: 1.2976167 (2406)	total: 12.4s	remaining: 5m 49s
2408:	learn: 1.4462285	test: 1.2976239	best: 1.2976167 (2406)	total: 12.5s	remaining: 5m 49s
2409:	learn: 1.4461890	test: 1.2976580	best: 1.2976167 (2406)	total: 12.5s	remaining: 5m 49s
2410:	learn: 1.4461746	test: 1.2976474	best: 1.2976167 (2406)	total: 1

2512:	learn: 1.4426585	test: 1.2947619	best: 1.2946333 (2506)	total: 13s	remaining: 5m 49s
2513:	learn: 1.4426133	test: 1.2947507	best: 1.2946333 (2506)	total: 13s	remaining: 5m 49s
2514:	learn: 1.4425858	test: 1.2947519	best: 1.2946333 (2506)	total: 13s	remaining: 5m 49s
2515:	learn: 1.4425368	test: 1.2947599	best: 1.2946333 (2506)	total: 13s	remaining: 5m 49s
2516:	learn: 1.4425123	test: 1.2945299	best: 1.2945299 (2516)	total: 13s	remaining: 5m 49s
2517:	learn: 1.4424770	test: 1.2945036	best: 1.2945036 (2517)	total: 13s	remaining: 5m 49s
2518:	learn: 1.4424416	test: 1.2945517	best: 1.2945036 (2517)	total: 13s	remaining: 5m 49s
2519:	learn: 1.4424148	test: 1.2945407	best: 1.2945036 (2517)	total: 13s	remaining: 5m 49s
2520:	learn: 1.4423765	test: 1.2945439	best: 1.2945036 (2517)	total: 13s	remaining: 5m 49s
2521:	learn: 1.4423408	test: 1.2945285	best: 1.2945036 (2517)	total: 13s	remaining: 5m 49s
2522:	learn: 1.4422738	test: 1.2945072	best: 1.2945036 (2517)	total: 13.1s	remaining: 5m 4

In [148]:
#training.to_pickle("pickled/training")
#training = pd.read_pickle("pickled/training")

#pickle.dump(cb_model, open( "pickled/cb_model", "wb"), protocol=4)

#cb_model = pickle.load( open( "pickled/cb_model", "rb" ) )

In [363]:
scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    scores[cb_features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

[('shop_item_minmax_mean', 28.32312821380092),
 ('shop_item_me', 20.82030028960799),
 ('shop_item_max_units_block', 12.174888682829213),
 ('item_block_units_rolling_3', 9.039122665768097),
 ('item_me', 4.321487073728777),
 ('item_block_units_rolling_6', 3.7114941120030243),
 ('item_max_units_block', 1.6725000018738125),
 ('cat_units', 1.6576808682343986),
 ('item_minmax_mean', 1.6430990348838077),
 ('shop_cat_units', 1.5790554217935397),
 ('category_share_of_total_units', 1.5358693258486822),
 ('category_me', 1.4905284230543536),
 ('shop_item_share_of_shop_units', 1.1361658463435504),
 ('cat_min_units_block', 1.125896645584796),
 ('item_min_units_block', 1.0394369858020363),
 ('item_share_of_total_units', 0.9867433879486104),
 ('shop_category_me', 0.9393294111164564),
 ('cat_max_units_block', 0.819741131509532),
 ('shop_share_of_units', 0.7304521764146239),
 ('shop_item_share_of_total_units', 0.7078996449012165),
 ('cat_minmax_mean', 0.6772754805219315),
 ('item_units', 0.5925809344128

In [364]:
cb_features = [item[0] for item in scores.items() if item[1] > 1]